<a href="https://colab.research.google.com/github/RaghadQadah/Automatic-Spam-Review-Detection/blob/main/predictionFinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [51]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**import library**

In [ ]:
import pickle
from pandas import DataFrame
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import string
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')
from sklearn.metrics import classification_report, confusion_matrix,accuracy_score,precision_score,recall_score,f1_score


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


**functions to extract features from**

In [ ]:
def clean(df, text, name):
    df[name] = df[text].str.lower()
    df[name] = df[name].str.replace('http\S+|www.\S+', '', case=False)
    df[name] = df[name].str.replace('[^\w\s]', '')
    df[name] = df[name].str.replace('\d+', '')
    df[name] = df[name].apply(lambda x: ' '.join([word for word in  str(x).split() if word not in (stop)]))
    df[name] = df[name].apply(lambda x: " ".join(x for x in  str(x).split() if len(x) > 3))
    return df
def word_count(df, text, name):
    df[name] = df[text].apply(lambda x: len(str(x).split(" ")))
    return df

def hashtags_count(df, text, name):
    df[name] = df[text].apply(lambda x: len([x for x in  str(x).split() if x.startswith('#')]))
    return df

def user_mentions_count(df, text, name):
    df[name] = df[text].apply(lambda x: len([x for x in  str(x).split() if x.startswith('@')]))
    return df

def numerics_count(df, text, name):
    df[name] = df[text].apply(lambda x: len([x for x in  str(x).split() if x.isdigit()]))
    return df

def links_count(df, text, name):
    df[name] = df[text].apply(
        lambda x: len([x for x in  str(x).split() if x.startswith(('httP', 'HTTPS', 'https', 'www', 'WWW', 'HTTP', 'http')) ]))
    return df

**upload Model**

In [ ]:
with open('drive/My Drive/model_pickle','rb') as file:
    mp = pickle.load(file)

In [ ]:
listTest=[["0gvJ0LlByoUsFi6x4eEQ","biZ0fIcZp7B-KBJ3EGcX7Q","I still love this place but the took Mu Shu off the menu. Â I am deeply saddened. Show owner comment Â»",5,2,"Update - 5/12/2009","biZ0fIcZp7B-KBJ3EGcX7Q",268,543],
      ["#NAME?","7dwh3pL2tbSeSvL0qLzUrw","The highlight is the unique menu. The assorted pancakes waffles and other breakfast goodness make it a great breakfast choice. The portions are generous the ingredients are fresh and the flavors are wonderful! The service and atmosphere are also both amazing. The one downside is that they really back you in here when they are busy but business is business!",4,1,"6/17/2011","7dwh3pL2tbSeSvL0qLzUrw",24,461]]

#listTest = ['item1', 'item2', 'item3',...]
df = DataFrame (listTest,columns=['reviewID', 'reviewerID', 'reviewContent', 'rating', 'usefulCount',
       'date', 'reviewerID.1', 'firstCount', 'reviewCount'])

df

,reviewID,reviewerID,reviewContent,rating,usefulCount,date,reviewerID.1,firstCount,reviewCount
0,0gvJ0LlByoUsFi6x4eEQ,biZ0fIcZp7B-KBJ3EGcX7Q,I still love this place but the took Mu Shu of...,5,2,Update - 5/12/2009,biZ0fIcZp7B-KBJ3EGcX7Q,268,543
1,#NAME?,7dwh3pL2tbSeSvL0qLzUrw,The highlight is the unique menu. The assorted...,4,1,6/17/2011,7dwh3pL2tbSeSvL0qLzUrw,24,461


In [ ]:
clean(df, "reviewContent", "cleanContent")

,reviewID,reviewerID,reviewContent,rating,usefulCount,date,reviewerID.1,firstCount,reviewCount,cleanContent
0,0gvJ0LlByoUsFi6x4eEQ,biZ0fIcZp7B-KBJ3EGcX7Q,I still love this place but the took Mu Shu of...,5,2,Update - 5/12/2009,biZ0fIcZp7B-KBJ3EGcX7Q,268,543,still love place took menu deeply saddened sho...
1,#NAME?,7dwh3pL2tbSeSvL0qLzUrw,The highlight is the unique menu. The assorted...,4,1,6/17/2011,7dwh3pL2tbSeSvL0qLzUrw,24,461,highlight unique menu assorted pancakes waffle...


**TF-IDF**

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidfconverter = TfidfVectorizer(max_features=1000,stop_words=stopwords.words('english'))
tfidf_matrix = tfidfconverter.fit_transform(df["cleanContent"])
#print(tfidf_matrix.toarray())

**calculateing cosine similarity**

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
cosineSim=[]
for i in range(len(tfidf_matrix.toarray())):
  sim=(cosine_similarity(tfidf_matrix[i], tfidf_matrix)).mean()
  cosineSim.append(sim)

In [ ]:
list_of_rows = [list(row) for row in df.values]
#print(len(cosineSim))
links_count(df,"reviewContent","linksCount")
hashtags_count(df, "reviewContent", "hashtagsCount")
user_mentions_count(df, "reviewContent", "userMentionsCount")
numerics_count(df,"reviewContent","numericsCount")
clean(df, "reviewContent", "cleanContent")

word_count(df, "reviewContent", "wCount")
word_count(df, "cleanContent", "wordCount")


,reviewID,reviewerID,reviewContent,rating,usefulCount,date,reviewerID.1,firstCount,reviewCount,cleanContent,linksCount,hashtagsCount,userMentionsCount,numericsCount,wCount,wordCount
0,0gvJ0LlByoUsFi6x4eEQ,biZ0fIcZp7B-KBJ3EGcX7Q,I still love this place but the took Mu Shu of...,5,2,Update - 5/12/2009,biZ0fIcZp7B-KBJ3EGcX7Q,268,543,still love place took menu deeply saddened sho...,0,0,0,0,21,10
1,#NAME?,7dwh3pL2tbSeSvL0qLzUrw,The highlight is the unique menu. The assorted...,4,1,6/17/2011,7dwh3pL2tbSeSvL0qLzUrw,24,461,highlight unique menu assorted pancakes waffle...,0,0,0,0,60,28


In [ ]:
dateList=[]
for word in list_of_rows:
    if "Update" in str(word[5]):
        dateList.append(1)
    else:
        dateList.append(0)

extremeRating=[]
for rating in list(df["rating"]):
    if rating == 5 or rating == 1:
        extremeRating.append(1)
    else:
        extremeRating.append(0)




dict = {'UpdateOrNot': dateList,
       'NumberOfURLs': list(df["linksCount"]),
       'NumberOfnumerics': list(df["numericsCount"]),
      #'hashtagsCount': list(df["hashtagsCount"]),
      #'userMentionsCount': list(df["userMentionsCount"]),
       'extremeRating':extremeRating,
       'firstCount':list(df["firstCount"]),
       'wordCount':list(df['wCount']),
       'reviewCount':list(df['reviewCount']),
       'usefulCount':list(df['usefulCount']),
        'cosineSimilarty':cosineSim,
       #'Filtered':list(df["Filtered"])
        }


df1 = pd.DataFrame(dict)
#df1.to_csv('Data.csv',index=False)
df1.head()


,UpdateOrNot,NumberOfURLs,NumberOfnumerics,extremeRating,firstCount,wordCount,reviewCount,usefulCount,cosineSimilarty
0,1,0,0,1,268,21,543,2,0.514626
1,0,0,0,0,24,60,461,1,0.514626


In [52]:
list_of_featurse = [list(row) for row in df1.values]


In [55]:
result=mp.predict(list_of_featurse)
for i in range(len(result)):
  if result[i]==0:
    print(result[i],"non spam")
  else:
    print(result[i],"spam")

0 non spam
1 spam
